# 5-7 优化器optimizers
深度学习优化算法大概经历了 SGD -> SGDM -> NAG ->Adagrad -> Adadelta(RMSprop) -> Adam -> Nadam 这样的发展历程。

一些爱写论文的炼丹师由于追求评估指标效果，可能会偏爱前期使用Adam优化器快速下降，后期使用SGD并精调优化器参数得到更好的结果。

此外目前也有一些前沿的优化算法，据称效果比Adam更好，例如LazyAdam, Look-ahead, RAdam, Ranger等。

## 一 优化器的使用
优化器主要是用apply_gradients方法传入变量和对应梯度从而来对给定变量进行迭代, 或者直接使用minimizer方法对目标函数进行迭代优化。

初始化优化器的时候会创建一个optimizer.iterarions用于记录迭代的次数。因此优化器和tf.Variable一样, 一般需要在@tf.function之外创建。

In [1]:
import tensorflow as tf
import numpy as np

In [15]:
# 求 f(x) = a*x**2 + b*x + c的最小值
x = tf.Variable(0.0, name = "x", dtype = tf.float32)
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.01)
a = tf.constant(1.0)
b = tf.constant(-2.0)
c = tf.constant(1.0)

@tf.function
def minimizef():
    with tf.GradientTape() as tape:
        y = a*x**2 + b*x + c
    dy_dx = tape.gradient(y, x)
    optimizer.apply_gradients(zip([dy_dx], [x]))

    if tf.math.mod(optimizer.iterations, 100) == 0:
        tf.print("iteration :", optimizer.iterations)
        tf.print("x = ", x)
    return a*x**2 + b*x + c

In [16]:
for _ in range(1000):
    minimizef()

iteration : 100
x =  0.867380381
iteration : 200
x =  0.98241204
iteration : 300
x =  0.997667611
iteration : 400
x =  0.999690711
iteration : 500
x =  0.999959
iteration : 600
x =  0.999994516
iteration : 700
x =  0.999998569
iteration : 800
x =  0.999998569
iteration : 900
x =  0.999998569
iteration : 1000
x =  0.999998569


## 二 内置优化器
深度学习优化算法大概经历了 SGD -> SGDM -> NAG ->Adagrad -> Adadelta(RMSprop) -> Adam -> Nadam 这样的发展历程。

在keras.optimizers子模块中，它们基本上都有对应的类的实现。

* SGD, 默认参数为纯SGD, 设置momentum参数不为0实际上变成SGDM, 考虑了一阶动量, 设置 nesterov为True后变成NAG，即 Nesterov Accelerated Gradient，在计算梯度时计算的是向前走一步所在位置的梯度。

* Adagrad, 考虑了二阶动量，对于不同的参数有不同的学习率，即自适应学习率。缺点是学习率单调下降，可能后期学习速率过慢乃至提前停止学习。

* RMSprop, 考虑了二阶动量，对于不同的参数有不同的学习率，即自适应学习率，对Adagrad进行了优化，通过指数平滑只考虑一定窗口内的二阶动量。

* Adadelta, 考虑了二阶动量，与RMSprop类似，但是更加复杂一些，自适应性更强。

* Adam, 同时考虑了一阶动量和二阶动量，可以看成RMSprop上进一步考虑了一阶动量。

* Nadam, 在Adam基础上进一步考虑了 Nesterov Acceleration。